# Demo notebook: Analysing battery data

## Introduction

In this notebook , we will extract battery data from the Aware platform and infer users' behavioral patterns from their interaction with the phone. The below functions will be described in this notebook:

- `niimpy.preprocessing.battery.battery_shutdown_info`: returns the timestamp when the device is shutdown or rebooted
- `niimpy.preprocessing.battery.battery_occurrences`: returns the number of battery samples within a time range
- `niimpy.preprocessing.battery.battery_gaps`: returns the time gaps between two battery sample


## Read data

In [1]:
import sys
sys.path.append('../../')

import pandas as pd
import niimpy
import niimpy.preprocessing.battery as battery
from niimpy.config import config

In [2]:
data = niimpy.read_csv(config.MULTIUSER_AWARE_BATTERY_PATH, tz='Europe/Helsinki')
data.shape

(505, 8)

In [3]:
data.head()

,user,device,time,battery_level,battery_status,battery_health,battery_adaptor,datetime
2020-01-09 02:20:02.924999936+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578529e+09,74,3,2,0,2020-01-09 02:20:02.924999936+02:00
2020-01-09 02:21:30.405999872+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578529e+09,73,3,2,0,2020-01-09 02:21:30.405999872+02:00
2020-01-09 02:24:12.805999872+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578529e+09,72,3,2,0,2020-01-09 02:24:12.805999872+02:00
2020-01-09 02:35:38.561000192+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578530e+09,72,2,2,0,2020-01-09 02:35:38.561000192+02:00
2020-01-09 02:35:38.953000192+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578530e+09,72,2,2,2,2020-01-09 02:35:38.953000192+02:00


# Feature extraction

The dataframe's index first needs to be sorted in ascending order to be compatible with the functions in this module.

First, we can count the amount the battery samples every 10 minutes by using the `battery_occurences` function. This function requires the index to be sorted.

In [4]:
data = data.sort_index()
occurences = battery.battery_occurrences(battery_data=data, hours=0, minutes=10)
occurences.head()

,start,end,occurrences
2019-08-05 14:00:58.600000+03:00,2019-08-05 14:00:58.600000+03:00,2019-08-05 14:10:58.600000+03:00,1.0
2019-08-05 14:10:58.600000+03:00,2019-08-05 14:10:58.600000+03:00,2019-08-05 14:20:58.600000+03:00,0.0
2019-08-05 14:20:58.600000+03:00,2019-08-05 14:20:58.600000+03:00,2019-08-05 14:30:58.600000+03:00,1.0
2019-08-05 14:30:58.600000+03:00,2019-08-05 14:30:58.600000+03:00,2019-08-05 14:40:58.600000+03:00,0.0
2019-08-05 14:40:58.600000+03:00,2019-08-05 14:40:58.600000+03:00,2019-08-05 14:50:58.600000+03:00,0.0


The above dataframe gives the battery information of all users. You can also get the information for an individual by passing a filter dataframe.

In [5]:
individual_occurences = battery.extract_features_battery(data.query('user == "jd9INuQ5BBlW"'), feature_functions={f: {'hours':0,'minutes':10}})
individual_occurences.head()

NameError: name 'f' is not defined

Next, you can extract the gaps between two consecutive battery samples with the `battery_gaps` function.

In [ ]:
f = niimpy.preprocessing.battery.battery_gaps
gaps = battery.battery_gaps(data)
gaps.head()[['user', 'device','delta']]

The most important piece of information is the  `delta` column, which gives the time gap between two battery samples.

Knowing when the phone is shutdown is essential if we want to infer the usage behaviour of the subjects. This can be done by calling the `shutdown_info` function. The function returns the timestamp when the phone is shut down or rebooted (e.g: battery_status = -1).

In [ ]:
shutdown = battery.shutdown_info(data, feature_functions={'battery_column_name': 'battery_status'})
shutdown

## Extracting featrues with the extract_features call

We have seen above how to extract battery features using `niimpy`. Sometimes, we need more than one features and it would be inconvenient to extract everthing one by one. `niimpy` provides a `extract_feature` call to allow you extracting all the features available and combining them into a single data frame. The extractable features must start with the prefix `battery_`.

In [ ]:
# Start by defining the feature name
f0 = niimpy.preprocessing.battery.battery_occurrences
f1 = niimpy.preprocessing.battery.battery_gaps
f2 = niimpy.preprocessing.battery.battery_charge_discharge

# The extract_feature function requires a feature_functions parameter. 
# This parameter accepts a dictionary where the key is the feature name and value
# is a dictionary containing values passed to the function.
features = battery.extract_features_battery(data, feature_functions={f0: {'hours':0,'minutes':10},
                                                                     f1: {},
                                                                    f2: {}})
features.head()